In [ ]:
import os
import sys

In [ ]:
IN_COLAB='google.colab' in sys.modules

In [ ]:
if IN_COLAB:
    from google.colab import drive

    WORKING_FOLDER="/content/drive/MyDrive/unicamp/ia368v_dd/trabalho_final_UNICAMP-IR/artigo/unicamp_IR"

    drive.mount('/content/drive', force_remount=True)

    os.chdir(WORKING_FOLDER)

    !pip install openai -q
    !pip install tiktoken -q
else:
    WORKING_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/unicamp_IR/"

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depende

In [ ]:
from common_tools import *
import tiktoken
import pandas as pd
import numpy as np
import json
from scipy.spatial import distance
import pickle
import time

In [ ]:
import glob

In [ ]:
API_FILES="../../../api_keys_20230612.json"

DATASET_1M="../../clueweb22-pt_10M_sample_fixed/clueweb22-pt_cleaned_massiveweb_1M_sample.tsv"
BM25_MT5_1M_RETRIEVAL="retrievals/200-queries-passagens-by-BM25-mT5_1M_dataset.tsv"

In [ ]:
def get_embeddings(text, tokenizer, openai=openai):
    response = openai.embeddings.create(input=[tokenizer.encode(text)], model="text-embedding-ada-002")

    return {"adav2_embedding" : np.array(response.data[0].embedding),
            "adav2_usage": response.usage}

In [ ]:
def get_embeddings_list(text_list, tokenizer, openai=openai):

    tokenized_text = [tokenizer.encode(text) for text in text_list]

    response = openai.embeddings.create(input=tokenized_text, model="text-embedding-ada-002")

    print("Tokenized texts={}, Returned embeddings={}".format(len(tokenized_text), len(response.data)))

    return {"adav2_embeddings" : np.array([each_response.embedding for each_response in response.data]),
            "adav2_usage": response.usage}

### Initialize openai interface and create tokenizer

In [ ]:
openai.api_key=json.load(open(API_FILES))["OPENAI_API_KEY"]

In [ ]:
tokenizer = tiktoken.get_encoding("cl100k_base")

### Read the 1M dataset

In [ ]:
dataset_df = pd.read_csv(DATASET_1M, sep='\t', header=None, names=['clueweb_id', 'passage', 'num_tokens', 'colbertx_id'])

In [ ]:
dataset_df.head()

,clueweb_id,passage,num_tokens,colbertx_id
0,clueweb22-pt0000-00-00003_1,"Se você precisar de ajuda, visite o website nacional sobre a COVID-19 ou ligue para a linha de apoio à COVID-19 808 24 24 24 Perguntas mais frequentes Posso viajar entre Sintra e Cascais? Quais são as restrições de viagem em Cascais? Qual o número de telefone de apoio para a COVID 19 em Cascais? Preciso utilizar máscara facial no transporte público em Cascais? A prática do distanciamento social é compulsória em Cascais? O que eu devo fazer caso apresente sintomas da COVID-19 quando chegar em Cascais? Última atualização: 25 Abr 2022 Aplicam-se exceções, para detalhes completos: European Union. Estamos trabalhando ininterruptamente para lhe trazer as últimas informações de viagem relacionadas à COVID-19. Esta informação é compilada a partir de fontes oficiais. Ao melhor de nosso conhecimento, está correta de acordo com a última atualização. Visite Avisos de Viagem Rome2rio para ajuda geral. Perguntas & Respostas Qual a maneira mais econômica de ir de Sintra para Cascais? Qual a maneira mais rápida de ir de Sintra para Cascais? Existe um ônibus direto entre Sintra e Cascais? Qual a distância entre Sintra e Cascais? Como viajar de Sintra para Cascais sem carro? Qual a distância entre Sintra e Cascais? Aonde pegar ônibus de Sintra para Cascais?",361,5
1,clueweb22-pt0000-00-00003_2,"Qual a parada final de Sintra para Cascais ônibus? Posso dirigir de Sintra para Cascais? Quais são as opções de acomodação perto de Cascais? Abrir visualização de mapa Distância: 12.2 km Duração 33 min Quais empresas operam serviços entre Sintra, Portugal e Cascais, Portugal? Operadores de ônibus Scott URB FlixBus Outros Operadoras BlaBlaCar Táxi de Sintra para Cascais Mais Perguntas e Respostas Existe opções de transporte compartilhado entre Sintra e Cascais? Cascais, Portugal Cascais MHC é uma vila portuguesa, sede do município (concelho) homónimo, o qual é parte do distrito e da área metropolitana de Lisboa. Em 2016, totalizava habitantes distribuídos por uma área de. O município subdivide-se em quatro freguesias (Alcabideche, Carcavelos e Parede, Cascais e Estoril e São Domingos de Rana). Apesar do seu estatuto de vil ... Coisas para fazer em Cascais Quinta da Regaleira O Palácio da Regaleira é o edifício principal e o nome mais comum do palácio da Quinta da Regaleira. Também é designado Palácio do Monteiro dos Milhões, denominação esta associada à alcunha do seu primeiro proprietário foi António Augusto que foi distinguido pelo rei Dom Carlos I em 16 de agosto de 1904 como barão de Almeida. Casino Estoril",382,6
2,clueweb22-pt0000-00-00003_3,"O Casino Estoril fica localizado no Estoril, a 18 km de Lisboa e a 20 km do Aeroporto de Lisboa. É hoje o maior casino da Europa e é gerido pela empresa Estoril-Sol SGPS S.A.. Cabo da Roca O Cabo da Roca é o ponto mais ocidental de Portugal continental, assim como da Europa continental. Situa-se na freguesia de Colares, concelho de Sintra e distrito de Lisboa. O local é visitável, não até ao extremo mas até uma zona à altitude de 140 m. O cabo forma o extremo ocidental da Serra de Sintra, precipitando-se sobre o Oceano Atlântico. As suas coordenadas geográficas são N 38º46'51\"", W 9º30'2\"". Lugares para ficar em Cascais €226 InterContinental Cascais-Estoril, an IHG Hotel 4.8 Exceptional €180 Hotel Cascais Miragem 4.8 Exceptional €133 Grande Real Villa Italia 4.8 Exceptional Rome2rio facilita as viagens de Sintra a Cascais.",255,7
3,clueweb22-pt0000-00-00003_5,Sintra para Lisboa Sintra para Óbidos Leiria Portugal Sintra para Colares Portugal Sintra para Coimbra Sintra para Assafora Sintra para Valado dos Frades Sintra para Centro Colombo Sintra para Bombarral Sintra para Santarém Portugal Sintra para Santa Iria da Azóia Sintra para Salamanca Castela e Leão Espanha Sintra para Autódromo Internacional do Algarve Sintra para Alenquer Portugal Sintra para Vila Nova de Milfontes Sintra para Penha Longa Resort Sintra Sintra para Alverca do Rib

#### Compute ADA v2 encoding cost estimate

In [ ]:
dataset_df['num_tokens'].sum() * 1e-3 * 1e-4

31.5986871

### Check BM25 + mT5 1M retrieval to use as comparison basis for ADA v2 embeddings

In [ ]:
bm25_mt5_df = pd.read_csv(BM25_MT5_1M_RETRIEVAL, sep='\t')

In [ ]:
bm25_mt5_df.head(10)

,query,doc_id,passage-by-bm25-mt5
0,Qual objetivo da Enel Brasil?,629557,"Inovação e Sustentabilidade inovação Cultura da Inovação Saiba mais sobre como a Cultura da Inovação é fundamental para transformar a inovação na Enel. Sustentabilidade Nossos Compromissos Nosso objetivo: deixar um mundo melhor e mais sustentável para as gerações futuras. Sustentabilidade Relatórios anuais Conheça nossos principais indicadores, estratégias, iniciativas e resultados. SUSTENTABILIDADE Nossas Iniciativas Referência em sustentabilidade, o Ecoenel alia gestão ambiental, social e tecnologia. Notícias Veja Mais Notícias Nossa Energia para um amanhã melhor Estamos realizando manutenções noturnas no Rio de Janeiro Unidos pela vacina: A Enel Brasil abraça essa causa! Conheça o novo site da Enel Brasil Selecione o seu Estado: RIO DE JANEIRO CEARÁ GOIÁS SÃO PAULO"
1,Qual objetivo da Enel Brasil?,1957679,"Inovação e Sustentabilidade inovação Cultura da Inovação Saiba mais sobre como a Cultura da Inovação é fundamental para transformar a inovação na Enel. Sustentabilidade Nossos Compromissos Nosso objetivo: deixar um mundo melhor e mais sustentável para as gerações futuras. Sustentabilidade Relatórios anuais Conheça nossos principais indicadores, estratégias, iniciativas e resultados. SUSTENTABILIDADE Nossas Iniciativas Referência em sustentabilidade, o Ecoenel alia gestão ambiental, social e tecnologia. Notícias Veja Mais Notícias Anunciamos pacote de obras para o Rio de Janeiro Estamos realizando manutenções noturnas no Rio de Janeiro Unidos pela vacina: A Enel Brasil abraça essa causa! Conheça o novo site da Enel Brasil Selecione o seu Estado: RIO DE JANEIRO CEARÁ GOIÁS SÃO PAULO"
2,Qual objetivo da Enel Brasil?,4306167,"Negócios Enel Brasil: inovação tecnológica e sustentabilidade ambiental Geração de energia No Brasil, desempenhamos papel de liderança no mercado de geração renovável, por meio da Enel Green Power Brasil, e também atuamos em geração convencional, com a empresa Enel Geração Fortaleza. Enel Green Power Brasil A Enel é a atual líder em geração de energia solar e um dos maiores players eólicos do país, por meio da subsidiária Enel Green Power Brasil. A EGPB gerencia ao todo uma capacidade renovável total de mais de 4,3 GW, dos quais mais de 1,8 GW são de fonte eólica, cerca de 1,2 GW são de fonte solar e cerca de 1,3 GW de hidro. Conheça a Enel Green Power. Enel Geração Fortaleza Controlada pela Enel, a Central Geradora Termelétrica Fortaleza S.A. (Enel Geração Fortaleza) tem sede no município de Caucaia, no Ceará, e integra o Complexo Industrial e Portuário do Pecém, localizado a 50 quilômetros da capital Fortaleza. Construída numa área de 70 mil metros quadrados, gera energia em um ciclo combinado de gás e vapor, com capacidade instalada de 326,6 MW, e conta com uma linha de transmissão de 1,2 quilômetro em alta-tensão (230kV)."
3,Qual objetivo da Enel Brasil?,858805,"Ao nosso conhecimento, não existem pesquisas brasileiras, com representatividade nacional, especificamente delineadas para examinar os comportamentos das faixas etárias mais velhas, em tempos da epidemia do coronavírus 2. A iniciativa ELSI-COVID-19, por meio de inquéritos telefônicos conduzidos entre os participantes do Estudo Longitudinal da Saúde dos Idosos Brasileiros (ELSI-Brasil) 16, visa a preencher essa lacuna. Nesta publicação apresentamos os resultados da primeira série dos inquéritos telefônicos da iniciativa ELSI-COVID-19, com o objetivo de estimar as prevalências de comportamentos preventivos (distanciamento social, uso de máscaras e higienização das mãos) e examinar a associação entre este distanciamento e características sociodemográficas, tais como idade, sexo, escolaridade, situação conjugal, arranjos domiciliares e área de residência. Um objetivo adicional das nossas análises foi examinar a associação entre o distanciamento social e a presença de sintomas relacionados à COVID-19. Métodos Delineamento do ELSI-Brasil"
4,Qual objetivo da Enel Brasil?,31418

### Check BM25+mT5 passage ID alignment with the "colbertx_id" field of the 1M dataset

In [ ]:
dataset_df[dataset_df['colbertx_id'] == 629557]['passage'] == bm25_mt5_df.iloc[0]['passage-by-bm25-mt5']

63520    True
Name: passage, dtype: bool

In [ ]:
dataset_df[dataset_df['colbertx_id'] == 629557]

,clueweb_id,passage,num_tokens,colbertx_id
63520,clueweb22-pt0000-12-20426_1,"Inovação e Sustentabilidade inovação Cultura da Inovação Saiba mais sobre como a Cultura da Inovação é fundamental para transformar a inovação na Enel. Sustentabilidade Nossos Compromissos Nosso objetivo: deixar um mundo melhor e mais sustentável para as gerações futuras. Sustentabilidade Relatórios anuais Conheça nossos principais indicadores, estratégias, iniciativas e resultados. SUSTENTABILIDADE Nossas Iniciativas Referência em sustentabilidade, o Ecoenel alia gestão ambiental, social e tecnologia. Notícias Veja Mais Notícias Nossa Energia para um amanhã melhor Estamos realizando manutenções noturnas no Rio de Janeiro Unidos pela vacina: A Enel Brasil abraça essa causa! Conheça o novo site da Enel Brasil Selecione o seu Estado: RIO DE JANEIRO CEARÁ GOIÁS SÃO PAULO",220,629557


In [ ]:
dataset_df[dataset_df['colbertx_id'] == 8698250]

,clueweb_id,passage,num_tokens,colbertx_id
869255,clueweb22-pt0001-82-12574_0,"Ateus: 15 famosos brasileiros que não acreditam em Deus Página inicial Listas Ateus: 15 famosos brasileiros que não acreditam em Deus Ateus: 15 famosos brasileiros que não acreditam em Deus Redação S 16/06/2018 Sem religiões Um dos assuntos que rendem polêmicas não só no país mas em todos os cantos com certeza são futebol e religião, onde muitos definem ambos como algo indiscutível. Principalmente a religião por se tratar de algo extremamente pessoal e individual, no qual cada um escolhe qual quer seguir, sem ter que fazer comparações entre elas e nem definir qual delas estaria certa. Existem ainda os ateus, ou seja, pessoas que não acreditam em nenhuma existência divina ou sequer seguem alguma religião. No Brasil, um país bem diverso, felizmente conseguimos encontrar aqui uma imensidão de diversidades na cultura, etnia, nas belezas naturais e inclusive também nas religiões. Porém infelizmente algumas vezes essa diversidade acaba desencadeando em conflitos, por conta da falta de respeito e empatia com às diferenças. E quando se trata dos artistas, por exemplo, alguns preferem não falar a respeito de sua religião para evitar essas divergências e julgamentos relacionando suas crenças aos seus trabalhos.",328,8698250


### Check if the resulting ADA v2 embedding makes sense using cosine distance

In [ ]:
positive_text = get_embeddings(dataset_df[dataset_df['colbertx_id'] == 629557]['passage'].to_numpy()[0], tokenizer)

In [ ]:
positive_text_02 = get_embeddings(bm25_mt5_df.iloc[1]['passage-by-bm25-mt5'], tokenizer)

In [ ]:
negative_text = get_embeddings(dataset_df[dataset_df['colbertx_id'] == 8698250]['passage'].to_numpy()[0], tokenizer)

In [ ]:
query = get_embeddings(bm25_mt5_df.iloc[0]['query'], tokenizer)

In [ ]:
distance.cosine(query["adav2_embedding"], np.array(positive_text["adav2_embedding"]))

0.13535598082277855

In [ ]:
distance.cosine(query["adav2_embedding"], negative_text["adav2_embedding"])

0.2632175458522312

In [ ]:
distance.cosine(query["adav2_embedding"], positive_text_02["adav2_embedding"])

0.1355508144417764

### Compute the ADA v2 embeddings by chunks, sending individual passages to the API

In [ ]:
TEMP_FILE="retrievals/adav2_temp_{:06}_{:06}.pkl"

In [ ]:
results=[]
chunk_size=5000
starting_position=5000

In [ ]:
for starting_position in list(range(14000, 1000000, chunk_size)):
    start_time = time.time()

    results=[]

    for i in range(chunk_size):
        embedding_result = get_embeddings(dataset_df.iloc[starting_position + i]['passage'], tokenizer)

        print("Embedding {}: {}, tokens={}, usage={}".format(starting_position + i,
                                                            dataset_df.iloc[starting_position + i]['clueweb_id'],
                                                            dataset_df.iloc[starting_position + i]['num_tokens'],
                                                            embedding_result['adav2_usage']))

        results.append(embedding_result)

        if i % 1000 == 0:
            with open(TEMP_FILE.format(starting_position, starting_position + chunk_size - 1), "wb") as output_file:
                pickle.dump(results, output_file, pickle.HIGHEST_PROTOCOL)


    with open(TEMP_FILE.format(starting_position, starting_position + chunk_size - 1), "wb") as output_file:
        pickle.dump(results, output_file, pickle.HIGHEST_PROTOCOL)

    print("Elapsed time: {}".format(time.time() - start_time))

Streaming output truncated to the last 5000 lines.
Embedding 39051: clueweb22-pt0000-07-13585_1, tokens=333, usage=Usage(prompt_tokens=333, total_tokens=333)
Embedding 39052: clueweb22-pt0000-07-13592_1, tokens=385, usage=Usage(prompt_tokens=385, total_tokens=385)
Embedding 39053: clueweb22-pt0000-07-13594_3, tokens=166, usage=Usage(prompt_tokens=166, total_tokens=166)
Embedding 39054: clueweb22-pt0000-07-13613_0, tokens=373, usage=Usage(prompt_tokens=373, total_tokens=373)
Embedding 39055: clueweb22-pt0000-07-13627_3, tokens=269, usage=Usage(prompt_tokens=269, total_tokens=269)
Embedding 39056: clueweb22-pt0000-07-13635_2, tokens=235, usage=Usage(prompt_tokens=235, total_tokens=235)
Embedding 39057: clueweb22-pt0000-07-13635_3, tokens=366, usage=Usage(prompt_tokens=366, total_tokens=366)
Embedding 39058: clueweb22-pt0000-07-13642_2, tokens=298, usage=Usage(prompt_tokens=298, total_tokens=298)
Embedding 39059: clueweb22-pt0000-07-13649_8, tokens=153, usage=Usage(prompt_tokens=153, tota

KeyboardInterrupt: 

### Compute the ADA v2 embeddings sending 1k passages per API call

In [ ]:
chunk_size=1000
starting_position=44000

In [ ]:
for starting_position in list(range(44000, 1000000, chunk_size)):
    start_time = time.time()

    results = get_embeddings_list(dataset_df.iloc[starting_position:(starting_position + chunk_size)]['passage'].to_numpy(),
                                  tokenizer)

    with open(TEMP_FILE.format(starting_position, starting_position + chunk_size - 1), "wb") as output_file:
        pickle.dump(results, output_file, pickle.HIGHEST_PROTOCOL)

    print("Elapsed time: {}".format(time.time() - start_time))

Tokenized texts=1000, Returned embeddings=1000
Elapsed time: 7.947092533111572
Tokenized texts=1000, Returned embeddings=1000
Elapsed time: 9.845900774002075
Tokenized texts=1000, Returned embeddings=1000
Elapsed time: 7.181149244308472
Tokenized texts=1000, Returned embeddings=1000
Elapsed time: 9.524534463882446
Tokenized texts=1000, Returned embeddings=1000
Elapsed time: 7.84736180305481
Tokenized texts=1000, Returned embeddings=1000
Elapsed time: 7.0034520626068115
Tokenized texts=1000, Returned embeddings=1000
Elapsed time: 7.476336240768433
Tokenized texts=1000, Returned embeddings=1000
Elapsed time: 7.496836423873901
Tokenized texts=1000, Returned embeddings=1000
Elapsed time: 7.853645324707031
Tokenized texts=1000, Returned embeddings=1000
Elapsed time: 7.291051149368286
Tokenized texts=1000, Returned embeddings=1000
Elapsed time: 9.775656461715698
Tokenized texts=1000, Returned embeddings=1000
Elapsed time: 7.10847544670105
Tokenized texts=1000, Returned embeddings=1000
Elapse

### Merge the embeddings

In [ ]:
embedding_files = np.sort(glob.glob("retrievals/adav2_temp_*.pkl"))

In [ ]:
embedding_files.shape

(965,)

In [ ]:
embedding_files[:10]

array(['retrievals/adav2_temp_000000_004999.pkl',
       'retrievals/adav2_temp_005000_009999.pkl',
       'retrievals/adav2_temp_010000_013999.pkl',
       'retrievals/adav2_temp_014000_018999.pkl',
       'retrievals/adav2_temp_019000_023999.pkl',
       'retrievals/adav2_temp_024000_028999.pkl',
       'retrievals/adav2_temp_029000_033999.pkl',
       'retrievals/adav2_temp_034000_038999.pkl',
       'retrievals/adav2_temp_039000_043999.pkl',
       'retrievals/adav2_temp_044000_044999.pkl'], dtype='<U39')

### Merge initial 9 files which contains a list of embeddings

In [ ]:
embeddings_part01_dfs = []

In [ ]:
for which_file in embedding_files[:9]:
    with open(which_file, "rb") as input_file:
        embeddings_result = pickle.load(input_file)

    embeddings_part01_dfs.append(pd.DataFrame(embeddings_result))

In [ ]:
embeddings_part01_df = pd.concat(embeddings_part01_dfs)

In [ ]:
embeddings_part01_df

,adav2_embedding,adav2_usage
0,"[-7.52143605495803e-05, 0.01183332409709692, 0.03261881694197655, -0.03531990200281143, -0.006617657840251923, 0.038766998797655106, 0.004820150323212147, -0.013048812747001648, 0.0024197218008339405, -0.00013264255539979786, -0.0028554326854646206, -0.005733374040573835, -0.014830241911113262, -0.024065379053354263, 0.00037461475585587323, 0.009106514975428581, 0.030303599312901497, -0.0060485005378723145, 0.010598542168736458, -0.005505068227648735, -0.0051642172038555145, 0.0019052295247092843, 0.012000533752143383, -0.018161579966545105, -0.018701797351241112, 0.011164484545588493, -0.017621362581849098, 0.008997185155749321, -0.010032600723206997, -0.030766643583774567, 0.022792011499404907, 0.0013392879627645016, -0.009498815052211285, -0.005463265813887119, -0.004302442539483309, -0.00019424095808062702, 0.01018694881349802, -0.00405162712559104, 0.021004149690270424, -0.0370691753923893, -0.004620784427970648, 0.01802009530365467, -0.012193468399345875, -0.009891115128993988, 0.014238575473427773, 0.03508837893605232, -0.010862220078706741, -0.009318742901086807, -0.03403367102146149, 0.019049080088734627, 0.008810681290924549, 0.006752712186425924, -0.013074536807835102, 0.005572595167905092, -0.016335131600499153, -0.0149974524974823, 0.0067141251638531685, 0.008399087935686111, 0.017685674130916595, -0.006212495267391205, 0.010926531627774239, -0.014315749518573284, -0.03117823600769043, 0.010585680603981018, 0.005717296153306961, -0.0063475496135652065, -0.0023039609659463167, 0.009717474691569805, -0.0006125674699433148, 0.023383677005767822, 0.025866102427244186, 0.0487738735973835, 0.012733685784041882, -0.0041352324187755585, 0.04290866106748581, -0.028682949021458626, 0.007177168503403664, -0.007087131962180138, 0.010939394123852253, 0.03256736695766449, 0.012187037616968155, -0.009936133399605751, -0.030406497418880463, -0.012585769407451153, 0.025480233132839203, 0.03897279500961304, 0.009685318917036057, 0.05854922905564308, -0.011994102969765663, -0.012778704054653645, -0.0019052295247092843, 0.026161937043070793, -0.00821258407086134, 0.0038426148239523172, 0.006990665104240179, 0.009273724630475044, -0.008309051394462585, 0.042780037969350815, -0.02550595812499523, 0.006804161239415407, ...]","Usage(prompt_tokens=361, total_tokens=361)"
1,"[0.0056585329584777355, -0.0007435029256157577, 0.027764175087213516, -0.005712391808629036, -0.03411950543522835, 0.04225217550992966, -0.010266820900142193, -0.001606504898518324, 0.007351716980338097, -0.005796546116471291, -0.008536608889698982, 0.010940054431557655, -0.03037632443010807, -0.030295535922050476, -0.0043827542103827, 0.01536320336163044, 0.025798331946134567, 0.0023462208919227123, 0.004682343453168869, 0.009344490244984627, -0.006880452856421471, 0.004773229826241732, 0.017436763271689415, -0.006745805963873863, -0.016952035948634148, 0.011088166385889053, -0.0044029513373970985, 0.00830097682774067, -0.008004753850400448, -0.02803346887230873, 0.022310979664325714, -0.0050256927497684956, -0.0022098910994827747, -0.012495225295424461, -0.0038037728518247604, -0.004948270972818136, 0.011269939132034779, -0.0030564828775823116, 0.01773298718035221, -0.01553824357688427, -0.013646455481648445, -0.007836445234715939, -0.01173447072505951, -0.01572674885392189, 0.004301966167986393, 0.03401178866624832, -0.01828503981232643, 0.0001847186213126406, -0.024519186466932297, 0.032449886202812195, -0.0030228211544454098, 0.04001703858375549, -0.011821991764008999, 0.008543341420590878, -0.008886690251529217, -0.02008930593729019, 0.0016309097409248352, 0.00492470758035779, 0.022109009325504303, -0.004376021679490805, 0.011633485555648804, -0.006079304032027721, -0.036623936146497726, 0.01219900231808424, 0.0029268853832036257, 0.0045005702413618565, 0.01328290905803442, -0.010818872600793839, 0.004810257814824581, 0.006887185387313366, 0.031211134046316147, 0.050330985337495804, 0.00904153473675251, 0.0021694970782846212, 0.053

In [ ]:
adav2_embedding = np.stack(embeddings_part01_df['adav2_embedding'].to_numpy())

In [ ]:
adav2_embedding.shape

(44000, 1536)

In [ ]:
with open("retrievals/adav2_embeddings.pkl", "wb") as output_file:
    pickle.dump(adav2_embedding, output_file, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open("retrievals/adav2_embeddings.pkl", "rb") as input_file:
    adav2_embedding = pickle.load(input_file)

### Merge the rest of the files

In [ ]:
embeddings_part02 = []

In [ ]:
for i, which_file in enumerate(embedding_files[9:]):
    with open(which_file, "rb") as input_file:
        embeddings_result = pickle.load(input_file)

    embeddings_part02.append(embeddings_result['adav2_embeddings'])

In [ ]:
len(embeddings_part02)

956

In [ ]:
adav2_embeddings_part02 = np.concatenate(embeddings_part02)

In [ ]:
adav2_embeddings_part02.shape

(956000, 1536)

In [ ]:
with open("retrievals/adav2_embeddings_part02.pkl", "wb") as output_file:
    pickle.dump(adav2_embeddings_part02, output_file, pickle.HIGHEST_PROTOCOL)

In [ ]:
adav2_embedding_all = np.concatenate([adav2_embedding, adav2_embeddings_part02])

In [ ]:
adav2_embedding_all.shape

(1000000, 1536)

In [ ]:
with open("retrievals/adav2_embeddings_all.pkl", "wb") as output_file:
    pickle.dump(adav2_embedding_all, output_file, pickle.HIGHEST_PROTOCOL)

### Now encode the queries

In [ ]:
queries_df = pd.read_csv("tests/all_queries.tsv", sep="\t")

In [ ]:
queries_df

,query_id,query
0,0,Qual a maior característica da flora brasileira?
1,1,Qual a maior característica da fauna brasileira?
2,2,Por que os países Guiana e Suriname não são filiados a Conmebol?
3,3,Por que a Bolívia tem duas capitais?
4,4,Qual o que significa decolonialidade?
5,5,O que é sistema Swift?
6,6,Por que a seleção uruguaia ostenta 4 estrelas em seu escudo?
7,7,Qual a importância da Semana de Arte Moderna de 1922?
8,8,Por que Johann Sebastian Bach é considerado um revolucionário na história da música?
9,9,quais países europeus seguem o regime monarquista?


In [ ]:
queries_embeddings = get_embeddings_list(queries_df['query'].to_numpy(), tokenizer)

Tokenized texts=200, Returned embeddings=200


In [ ]:
queries_embeddings

{'adav2_embeddings': array([[ 0.00996193,  0.02128673,  0.02067668, ..., -0.00608099,
         -0.02410332, -0.03862762],
        [ 0.01637173,  0.02459634,  0.0114912 , ..., -0.01240791,
         -0.01229171, -0.02905079],
        [ 0.01036395,  0.00372544,  0.0225684 , ..., -0.00121299,
         -0.00621329,  0.0023641 ],
        ...,
        [-0.00131368,  0.01733864,  0.01289437, ...,  0.00609177,
          0.00111522, -0.02650619],
        [-0.01645008,  0.01184985,  0.02236556, ...,  0.01767093,
         -0.01184355, -0.03393222],
        [-0.00779294,  0.01908889,  0.03236481, ...,  0.02419111,
         -0.01828929, -0.01311092]]),
 'adav2_usage': Usage(prompt_tokens=2709, total_tokens=2709)}

In [ ]:
queries_embeddings['adav2_embeddings'].shape

(200, 1536)

In [ ]:
with open("retrievals/adav2_query_embeddings.pkl", "wb") as output_file:
    pickle.dump(queries_embeddings['adav2_embeddings'], output_file, pickle.HIGHEST_PROTOCOL)